In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [ ]:
import p5_util

import P7_DataBreed
oP7_DataBreed = p5_util.object_load('./data/oP7_DataBreed.dump')
oP7_DataBreed.show()
oP7_DataBreed.breed_show()


# Building a new model from pre-trained VGG16

`VGG16` is a CNN network provided by / implemented in `Keras` with 16 layers.

In [ ]:
from keras.applications.vgg16 import VGG16
import p5_util

model = VGG16() 

if False :
    filename = './data/vgg16_pretrained.dump'
    p5_util.object_dump(model,filename)

In [ ]:
type(model)

#### A breed and an image is selected and displayed

In [ ]:
oP7_DataBreed.breed_show()

In [ ]:
import P7_DataBreed

oP7_DataBreed.show_image_name('Mexican_hairless')

In [ ]:
import cv2

import p3_util
import p3_util_plot


import P7_DataBreed
import p7_util

breedname='Doberman'
imagename='n02107142_16400.jpg'
imagename='n02107142_4314.jpg'

oP7_DataBreed_single = P7_DataBreed.P7_DataBreed()
oP7_DataBreed_single._dict_breed_sample=oP7_DataBreed._dict_breed_sample.copy()

list_restricted_image = [(breedname,[imagename])]
oP7_DataBreed_single.list_restricted_image = list_restricted_image

oP7_DataBreed_single.load(dirbreed='n02107142-Doberman', imagename=imagename)

p7_util.p7_image_pil_show(oP7_DataBreed_single._dict_img_pil,std_image_size=(500,375), is_title=False)

In [ ]:
from keras.preprocessing.image import load_img, img_to_array
from keras.applications import vgg16


image_pathname = oP7_DataBreed._build_pathname(dirbreed,imagename)

# Charger l'image PIL
pil_image = load_img(image_pathname, target_size=(224, 224))  

# Convertir en tableau numpy
arr_image = img_to_array(pil_image)  

# Créer la collection d'images (un seul échantillon)
arr_image = img.reshape((1, arr_image.shape[0], arr_image.shape[1], arr_image.shape[2]))  
X_test = vgg16.preprocess_input(arr_image)  # Prétraiter l'image comme le veut VGG-16

y_pred = model.predict(X_test)


# Afficher les 3 classes les plus probables
[print(my_tuple) for my_tuple in vgg16.decode_predictions(y_pred, top=3)[0]]

oP7_DataBreed.breed_show()


In [ ]:
vgg16.decode_predictions(y_pred, top=3)[0]

In [ ]:
#print('Top 3 :', vgg16.decode_predictions(y_pred, top=3)[0])
print('Top 3 : {}'.format([my_tuple for my_tuple in vgg16.decode_predictions(y_pred, top=3)[0]]))

# Transfer Learning from `VGG16`

In [ ]:
import p5_util

import P7_DataBreed
oP7_DataBreed = p5_util.object_load('./data/oP7_DataBreed.dump')
oP7_DataBreed.show()
oP7_DataBreed.breed_show()

In [ ]:
nbreed = oP7_DataBreed.sampling_breed_count
nbreed

#### Download a VGG16 model that has been pre-trained with imageNet  : 
* last fully connected layers are removed; in the resulting model only convolutional layers stay.
* Image reshape is enforced to (224,224,3)

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.layers import Dense
import p5_util
if False :
    # Load ImageNet pre-trained model from VGG-16 
    # Option include_top=False leads to remove last Fully connected layer.
    # VGG16 model is downloaed from network.
    vgg16_pretrained_model = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
    p5_util.object_dump(vgg16_model,'./data/vgg16_pretrained_model.dump')
else :
    vgg16_pretrained_model = p5_util.object_load('./data/vgg16_pretrained_model.dump')

#### Transfer learning model is built
* Output of all layers previous top layer are extracted.
* A new fully connected layer (dense layer) is built using previous layer as an input layer

In [ ]:
from keras import models
from keras import layers

# Outputs parameters are retrieved from previous layers
vgg16_model_tensor = vgg16_pretrained_model.output

# A new fully connected layer is then added in order to classify nbreeds breeds.
# Activation function is Softmax; This last layer is applied to the data input vgg16_model_parameters
vgg16_fc_tensor = layers.Dense(nbreed, activation='softmax')(vgg16_model_tensor)

# New VGG16 model for transfer learning is defined with : 
# * All layers except last layer are issued from 
# * fully connected layer previously defined.
model_vgg16_transfer_learning = models.Model(inputs=vgg16_pretrained_model.input, outputs=vgg16_fc_tensor)
type(model_vgg16_transfer_learning)

#### Compilation : shape of all internal layers are computed by compiler

In [ ]:
# Compiler le modèle 
from keras import optimizers

model_vgg16_transfer_learning.compile(loss="categorical_crossentropy", optimizer=optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

In [ ]:
import numpy as np
import p5_util

filename='./data/arr_keras_X_y_train_test.dump'
(X_train,X_test, y_train, y_test) = p5_util.object_load(filename)
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)
if False :
    n_xtrain = int((X_train.shape[0]*X_train.shape[1]*X_train.shape[2]*X_train.shape[3])/(7*7*512))
    print(n_xtrain)

    n_ytrain = int((y_train.shape[0]*y_train.shape[1])/(7*7*512))
    print(n_ytrain)

    X_train = X_train.reshape((n_xtrain, 7, 7, 512))
    #y_train = y_train.reshape((n_ytrain, 7, 7, 512))
    print(X_train.shape, y_train.shape)
    np.unique(y_train)

In [ ]:
n_ytrain = int((y_train.shape[0]*y_train.shape[1])/(7*7*3))
print(n_ytrain)
y_train.shape,7*7*3

In [ ]:
#help(model_vgg16_transfer_learning.fit)
model_vgg16_transfer_learning.summary()

In [ ]:
# Entraîner sur les données d'entraînement (X_train, y_train)
model_info = model_vgg16_transfer_learning.fit(X_train, y_train, epochs=1, batch_size=None, verbose=2)

## Total fine-tuning

On remplace la dernière couche fully-connected du réseau pré-entraîné par un classifieur adapté au nouveau problème (SVM, régression logistique...) et initialisé de manière aléatoire. Toutes les couches sont ensuite entraînées sur les nouvelles images. 

La stratégie #1 doit être utilisée lorsque la nouvelle collection d'images est grande : dans ce cas, on peut se permettre d'entraîner tout le réseau sans courir le risque d'overfitting. De plus, comme les paramètres de toutes les couches (sauf de la dernière) sont initialement ceux du réseau pré-entraîné, la phase d'apprentissage sera faite plus rapidement que si l'initialisation avait été aléatoire.